In [ ]:
import pandas as pd

# Caricamento dei dati
bassa = pd.read_csv('BES_Istat/classeBenessereBASSA.csv')
medio_bassa = pd.read_csv('BES_Istat/classeBenessereMEDIO-BASSA.csv')
media = pd.read_csv('BES_Istat/classeBenessereMEDIA.csv')
medio_alta = pd.read_csv('BES_Istat/classeBenessereMEDIO-ALTA.csv')
alta = pd.read_csv('BES_Istat/classeBenessereALTA.csv')

# Manipola i dati per convertire le percentuali in formato corretto (float)
def convert_percent(x):
    return float(x.replace(",", "."))

bassa["value"] = bassa["value"].apply(convert_percent)
medio_bassa["value"] = medio_bassa["value"].apply(convert_percent)
media["value"] = media["value"].apply(convert_percent)
medio_alta["value"] = medio_alta["value"].apply(convert_percent)
alta["value"] = alta["value"].apply(convert_percent)

# Unisci i dati dei vari DataFrame in uno
merged_classiBes = pd.merge(bassa, medio_bassa, on="prov_name", suffixes=("_bassa", "_mediobassa"))
merged_classiBes = pd.merge(merged_classiBes, media, on="prov_name")
merged_classiBes = pd.merge(merged_classiBes, medio_alta, on="prov_name", suffixes=("_media", "_medioalta"))
merged_classiBes = pd.merge(merged_classiBes, alta, on="prov_name")

# Calcola l'indice di qualità della vita
merged_classiBes["quality_index"] = (
    100 - merged_classiBes["value_bassa"] - merged_classiBes["value_mediobassa"] +
    merged_classiBes["value_media"] + merged_classiBes["value_medioalta"] + merged_classiBes["value"]
)

# Normalizza l'indice di qualità della vita in un intervallo da 0 a 100
min_value = merged_classiBes["quality_index"].min()
max_value = merged_classiBes["quality_index"].max()
merged_classiBes["normalized_quality_index"] = 25 + 50 * (merged_classiBes["quality_index"] - min_value) / (max_value - min_value)

# Mostra il DataFrame risultante
print(merged_classiBes[["prov_name", "quality_index", "normalized_quality_index"]])
merged_classiBes[["prov_name", "quality_index", "normalized_quality_index"]].to_csv("BES_Istat/IdQ BES.csv", index=False)

                 prov_name  quality_index  normalized_quality_index
0                Agrigento           78.8                 29.484305
1    Barletta-Andria-Trani          128.8                 51.905830
2                   Varese          150.7                 61.726457
3                  Venezia          147.5                 60.291480
4     Verbano-Cusio-Ossola          137.6                 55.852018
..                     ...            ...                       ...
102                Trapani          100.0                 38.991031
103                 Trento          180.3                 75.000000
104                Treviso          141.1                 57.421525
105                Trieste          170.0                 70.381166
106                  Udine          159.9                 65.852018

[107 rows x 3 columns]


In [ ]:
import pandas as pd

# Caricamento dei dati
classificaSole = pd.read_csv('Classifica_Sole24/classificaSole24.csv', sep=';')

# Manipola i dati per convertire le percentuali in formato corretto (float)
def convert_percent(x):
    return float(x.replace(",", "."))

classificaSole["Punteggio"] = classificaSole["Punteggio"].apply(convert_percent)

# Normalizza l'indice di qualità della vita in un intervallo da 0 a 100
min_value = classificaSole["Punteggio"].min()
max_value = classificaSole["Punteggio"].max()
classificaSole["normalized_Punteggio"] = 25 + 50 * (classificaSole["Punteggio"] - min_value) / (max_value - min_value)

# Mostra il DataFrame risultante
print(classificaSole)
classificaSole.to_csv("Classifica_Sole24/IdQ Sole.csv", index=False)

         Provincia  Punteggio  normalized_Punteggio
0            Udine     605.68             75.000000
1          Bologna     598.24             73.174950
2           Trento     597.09             72.892852
3            Aosta     594.21             72.186381
4          Bergamo     592.33             71.725212
..             ...        ...                   ...
102        Crotone     415.40             28.323848
103       Siracusa     415.40             28.323848
104         Napoli     412.22             27.543786
105  Caltanissetta     404.81             25.726095
106         Foggia     401.85             25.000000

[107 rows x 3 columns]


I due file csv vengono corretti manualmente per far combaciare i nomi delle province

In [ ]:
import pandas as pd

# Caricamento dei dati
IdQ_BES = pd.read_csv('IdQ BES_Correct.csv')
IdQ_Sole = pd.read_csv('IdQ Sole_Correct.csv')

# Unione dei due DataFrame in base al nome della provincia
merged_IdQ = pd.merge(IdQ_BES, IdQ_Sole, left_on="prov_name", right_on="Provincia", how="inner")

# Calcola la media tra "normalized_Punteggio" e "normalized_quality_index"
merged_IdQ["IdQ"] = (merged_IdQ["normalized_quality_index"] + merged_IdQ["normalized_Punteggio"]*2) / 3

# Crea un nuovo DataFrame con le colonne desiderate
finalResult = merged_IdQ[["Provincia", "IdQ"]]

# Salva il nuovo DataFrame in un file CSV
print(finalResult)
finalResult.to_csv("IdQ_Province.csv", index=False)

                 Provincia        IdQ
0                Agrigento  32.705827
1    Barletta-Andria-Trani  43.064425
2                   Varese  61.216379
3                  Venezia  60.845987
4     Verbano-Cusio-Ossola  56.126538
..                     ...        ...
102                Trapani  34.712002
103                 Trento  73.595235
104                Treviso  63.369359
105                Trieste  69.587881
106                  Udine  71.950673

[107 rows x 2 columns]
